In [62]:
import sys
import os
# sys.path.append('../')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import radio
from radio import batchflow as bf
from radio import CTImagesMaskedBatch as CTIMB
from radio.batchflow import FilesIndex, Dataset

In [60]:
DIR_CANCER = 'data/lunaset_split/train/cancer/*'
DIR_NCANCER = 'data/lunaset_split/train/ncancer/*'

cancerous_folder, non_cancerous_folder = 'data/lunaset_split/train/cancer', 'data/lunaset_split/train/noncancer'

In [64]:
cancer_index = FilesIndex(path=os.path.join(cancerous_folder, '*'), dirs=True)
non_cancer_index = FilesIndex(path=os.path.join(non_cancerous_folder, '*'), dirs=True)
cancer_set = Dataset(cancer_index, batch_class=CTIMB)
non_cancer_set = Dataset(non_cancer_index, batch_class=CTIMB)

In [66]:
from radio.pipelines import combine_crops
crops_sampling = combine_crops(cancer_set, non_cancer_set, batch_sizes=(5, 5))

In [67]:
np.random.seed(42)
batch_crops = crops_sampling.next_batch()
# wow! This works really fast!

StopIteration: 

In [54]:
from radio.pipelines import combine_crops
from utils import show_slices # function for plotting batch masks and images
combine_pipeline = combine_crops(cancerset, ncancerset, batch_sizes=(4, 4))

In [55]:
combine_pipeline

In [59]:
# Here we generate one batch with 4 cancer crops and 4 non-cancer crops
batch = combine_pipeline.next_batch(2)
show_slices(batch, scan_indices=4, ns_slice=16, clims=[(0, 255), (0, 1)],
            components=('images', 'masks'), grid=False)

StopIteration: 

In [18]:
from radio.models import Keras3DUNet
from radio.models.keras.losses import dice_loss

unet_config = dict(
    input_shape = (1, 32, 64, 64),
    num_targets = 1,
    loss= dice_loss
)

Using TensorFlow backend.


In [46]:
from radio.batchflow import F

train_unet_pipeline = (
    combine_crops(cancerset, ncancerset, batch_sizes=(4, 4))
    .init_model(
        name='3dunet', model_class=Keras3DUNet,
        config=unet_config, mode='static'
    )
    .train_model(
        name='3dunet',
        x=F(CTIMB.unpack, component='images', data_format='channels_first'),
        y=F(CTIMB.unpack, component='masks', data_format='channels_first')
    )
)

In [47]:
train_unet_pipeline.run()

In [48]:
keras_unet = train_unet_pipeline.get_model_by_name('3dunet')
keras_unet.save('3d_unet/weights')

In [49]:
unet_config.update({'path': '3d_unet/weights'})

train_unet_pipeline = (
    combine_crops(cancerset, ncancerset, batch_sizes=(4, 4))
    .init_model(
        name='3dunet', model_class=Keras3DUNet,
        config=unet_config, mode='static'
    )
    .train_model(
        name='3dunet',
        x=F(CTIMB.unpack, component='images', data_format='channels_first'),
        y=F(CTIMB.unpack, component='masks', data_format='channels_first')
    )
)

train_unet_pipeline.run()

In [50]:
from radio.batchflow.models.tf import VNet
# We use Tversky loss here cause it performs slightly better than dice
from radio.models.tf.losses import tversky_loss
import tensorflow as tf

In [51]:
vnet_config = {
    'inputs': dict(
        images={'shape': (32, 64, 64, 1)},
        labels={'name': 'targets', 'shape': (32, 64, 64, 1)}
    ),
    'loss': tversky_loss,
    'optimizer': 'Adam',
}
vnet_config['input_block/inputs'] = 'images'
vnet_config['head/layout'] = 'cna'
vnet_config['head/activation'] = tf.nn.sigmoid
vnet_config['head/kernel_size'] = 1
vnet_config['head/num_classes'] = 1

In [26]:
from IPython.display import clear_output
from radio.batchflow import V, F

train_vnet_pipeline = (
    combine_crops(cancerset, ncancerset, batch_sizes=(4, 4))
    .init_variable('loss_history', init_on_each_run=list)
    .init_variable('current_loss', 0)
    .init_model(
        name='vnet', model_class=VNet,
        config=vnet_config, mode='static'
    )
    .train_model(
        name='vnet', fetches='loss', save_to=V('current_loss'), mode='w',
        feed_dict={'images': F(CTIMB.unpack, component='images'),
                   'targets': F(CTIMB.unpack, component='masks')}
    )
    .print("Current loss:")
    .print(V('current_loss'))
    .update_variable('loss_history', value=V('current_loss'), mode='a')
    .call(lambda x: clear_output(wait=True))
)

ValueError: initial_block/inputs should be specified with a name or a list of names.